In [1]:
import os

import numpy as np
import pandas as pd

import xarray as xr

import xesmf as xe

In [1]:
import os

import numpy as np
import pandas as pd

import xarray as xr

import xesmf as xe

path_d = "../results/"
path_r = "../temp/NSRDB/"
names = [ "NSRDB_4km.nc", "NSRDB_2km.nc" ]
with xr.open_dataset(path_d + names[0]) as ds_4:
    with xr.open_dataset(path_d + names[1]) as ds_2:
        #ds_4 = ds_4.sel(ds_4["time"].dt.year.isin(
        #    np.unique( ds_2["time"].dt.year.values ) ))
        ds_4["Temperature"] += 273.15
        ds_4["Dew Point"] += 273.15
        ds_2["Temperature"] += 273.15
        ds_2["Dew Point"] += 273.15
        ds_2 = ds_2.drop_vars("Cloud Type")#

        # Cambiamos la resolución.
        regridder = xe.Regridder( ds_4, ds_2, "conservative" )
        ds_2_int = regridder( ds_4, keep_attrs = True )

        v_uv = [ "Global Horizontal UV Irradiance (280-400nm)",
            "Global Horizontal UV Irradiance (295-385nm)" ]

        f_int = ds_2 / ds_2_int.drop_vars( v_uv )
        f_int = f_int.where( f_int.apply(np.isfinite) & ~f_int.isnull(), 1 )
        f_int = f_int.mean( "time", skipna = True )
        f_int[ v_uv[0] ] = f_int["GHI"]
        f_int[ v_uv[1] ] = f_int["GHI"]
        f_int.to_netcdf(path_d + "NSRDB_f_int.nc")

        ds_2_corr = ds_2_int * f_int
        ds_2_corr["UVHI"] = ds_2_corr[ v_uv[0] ] + ds_2_corr[ v_uv[1] ]
        ds_2_corr = ds_2_corr.rename_vars( {
            "Relative Humidity": "Relative_Humidity",
            "Wind Speed": "Wind_Speed", "Wind Direction": "Wind_Direction",
            "Dew Point": "Dew_Point", "Surface Albedo": "Surface_Albedo",
            "Precipitable Water": "Precipitable_Water",
            "Solar Zenith Angle": "Solar_Zenith_Angle",
            v_uv[0]: "UV_A", v_uv[1]: "UV_B" } )
        
        f_regrid = "../temp/WRF_miroc_1985_2014_00054.nc"
        ds_rg = xr.open_dataset(f_regrid).drop_dims("bnds")
        ds_rg = ds_rg.isel( { "south_north": slice(1, 4),
            "west_east": slice(2, 5) } )
        ds_rg_2 = ds_rg.copy()
        ds_rg["XLAT"] = ds_rg["XLAT"].isel({"west_east": 0})
        ds_rg["XLONG"] = ds_rg["XLONG"].isel({"south_north": 0})
        ds_rg = ds_rg.rename_vars( { "XLAT": "lat",
            "XLONG": "lon", "XTIME": "time" } )
        ds_rg = ds_rg.swap_dims( { "south_north": "lat", 
            "west_east": "lon", "XTIME": "time" } )
        
        # Cambiamos la resolución.
        regridder = xe.Regridder( ds_2_corr, ds_rg, "conservative" )
        ds_2_rg = regridder( ds_2_corr, keep_attrs = True )

        ds_f = ds_rg_2.reindex( { "XTIME": ds_2_rg["time"].values }
            ).drop_vars("Dew_Point")
        dims = ["XTIME", "south_north", "west_east"]

        vars = list(ds_2_rg.keys())

        for v in vars: ds_f[v] = ( dims, ds_2_rg[v].values )
        ds_f[[v]].to_netcdf( path_r + "quantile_prep/NSRDB_int_001.nc" ) 

        for i in ["UV_A", "UV_B", "Solar_Zenith_Angle", "Dew_Point", "DHI"]:
            vars.remove(i)

        for v in vars:
            if not os.path.exists( path_r + "quantile_vars/" + v + "/" ):
                os.mkdir( path_r + "quantile_vars/" + v + "/" )
            ds_f[[v]].to_netcdf( path_r + "quantile_vars/"
                + v + "/WRF_miroc_1985_2014_00054.nc" ) 

In [2]:
path_d = "../results/"
path_r = "../temp/NSRDB/"
names = [ "NSRDB_4km.nc", "NSRDB_2km.nc" ]
ds_4 = xr.open_dataset(path_d + names[0])
ds_2 = xr.open_dataset(path_d + names[1])
#ds_4 = ds_4.sel(ds_4["time"].dt.year.isin(
#    np.unique( ds_2["time"].dt.year.values ) ))
ds_4["Temperature"] += 273.15
ds_4["Dew Point"] += 273.15
ds_2["Temperature"] += 273.15
ds_2["Dew Point"] += 273.15
ds_2 = ds_2.drop_vars("Cloud Type")#

In [3]:
# Cambiamos la resolución.
regridder = xe.Regridder( ds_4, ds_2, "conservative" )
ds_2_int = regridder( ds_4, keep_attrs = True )
ds_2_int

<xarray.Dataset> Size: 394MB
Dimensions:                                      (time: 219144, lat: 4, lon: 4)
Coordinates:
  * time                                         (time) datetime64[ns] 2MB 19...
  * lon                                          (lon) float64 32B -99.17 ......
  * lat                                          (lat) float64 32B 19.38 ... ...
Data variables: (12/14)
    Temperature                                  (time, lat, lon) float64 28MB ...
    GHI                                          (time, lat, lon) float64 28MB ...
    DNI                                          (time, lat, lon) float64 28MB ...
    Relative Humidity                            (time, lat, lon) float64 28MB ...
    Pressure                                     (time, lat, lon) float64 28MB ...
    Wind Speed                                   (time, lat, lon) float64 28MB ...
    ...                                           ...
    DHI                                          (time, lat, lon) float64 28MB ...
    Solar Zenith Angle                           (time, lat, lon) float64 28MB ...
    Surface Albedo                               (time, lat, lon) float64 28MB ...
    Precipitable Water                           (time, lat, lon) float64 28MB ...
    Global Horizontal UV Irradiance (280-400nm)  (time, lat, lon) float64 28MB ...
    Global Horizontal UV Irradiance (295-385nm)  (time, lat, lon) float64 28MB ...
Attributes:
    CDI:            Climate Data Interface version 2.4.0 (https://mpimet.mpg....
    Conventions:    CF-1.6
    history:        Thu Apr 04 12:36:16 2024: cdo -s -O collgrid temp/NSRDB_p...
    CDO:            Climate Data Operators version 2.4.0 (https://mpimet.mpg....
    regrid_method:  conservative

In [4]:
v_uv = [ "Global Horizontal UV Irradiance (280-400nm)",
    "Global Horizontal UV Irradiance (295-385nm)" ]

f_int = ds_2 / ds_2_int.drop_vars( v_uv )
f_int = f_int.where( f_int.apply(np.isfinite) & ~f_int.isnull(), 1 )
f_int = f_int.mean( "time", skipna = True )
f_int[ v_uv[0] ] = f_int["GHI"]
f_int[ v_uv[1] ] = f_int["GHI"]
f_int.to_netcdf(path_d + "NSRDB_f_int.nc")

f_int

<xarray.Dataset> Size: 2kB
Dimensions:                                      (lat: 4, lon: 4)
Coordinates:
  * lon                                          (lon) float64 32B -99.17 ......
  * lat                                          (lat) float64 32B 19.38 ... ...
Data variables: (12/14)
    Temperature                                  (lat, lon) float64 128B 1.00...
    GHI                                          (lat, lon) float64 128B 0.98...
    DNI                                          (lat, lon) float64 128B 0.98...
    Relative Humidity                            (lat, lon) float64 128B 0.99...
    Pressure                                     (lat, lon) float64 128B 1.00...
    Wind Speed                                   (lat, lon) float64 128B 1.02...
    ...                                           ...
    Dew Point                                    (lat, lon) float64 128B 1.0 ...
    Solar Zenith Angle                           (lat, lon) float64 128B 0.99...
    Surface Albedo                               (lat, lon) float64 128B 0.97...
    Precipitable Water                           (lat, lon) float64 128B 1.01...
    Global Horizontal UV Irradiance (280-400nm)  (lat, lon) float64 128B 0.98...
    Global Horizontal UV Irradiance (295-385nm)  (lat, lon) float64 128B 0.98...

In [5]:
ds_2_corr = ds_2_int * f_int
ds_2_corr["UVHI"] = ds_2_corr[ v_uv[0] ] + ds_2_corr[ v_uv[1] ]
ds_2_corr = ds_2_corr.rename_vars( { "Relative Humidity": "Relative_Humidity",
    "Wind Speed": "Wind_Speed", "Wind Direction": "Wind_Direction",
    "Dew Point": "Dew_Point", "Surface Albedo": "Surface_Albedo",
    "Precipitable Water": "Precipitable_Water",
    "Solar Zenith Angle": "Solar_Zenith_Angle",
    v_uv[0]: "UV_A", v_uv[1]: "UV_B" } )

ds_2_corr

<xarray.Dataset> Size: 423MB
Dimensions:             (time: 219144, lon: 4, lat: 4)
Coordinates:
  * time                (time) datetime64[ns] 2MB 1998-01-01 ... 2022-12-31T2...
  * lon                 (lon) float64 32B -99.17 -99.15 -99.13 -99.11
  * lat                 (lat) float64 32B 19.38 19.4 19.42 19.44
Data variables: (12/15)
    Temperature         (time, lat, lon) float64 28MB 284.7 284.5 ... 289.5
    GHI                 (time, lat, lon) float64 28MB 0.0 0.0 0.0 ... 78.7 75.21
    DNI                 (time, lat, lon) float64 28MB 0.0 0.0 ... 1.11e+03
    Relative_Humidity   (time, lat, lon) float64 28MB 48.78 49.8 ... 52.66 52.03
    Pressure            (time, lat, lon) float64 28MB 789.9 788.0 ... 786.1
    Wind_Speed          (time, lat, lon) float64 28MB 1.738 1.7 ... 0.9669
    ...                  ...
    Solar_Zenith_Angle  (time, lat, lon) float64 28MB 95.29 95.3 ... 82.4 82.51
    Surface_Albedo      (time, lat, lon) float64 28MB 0.09767 0.1 ... 0.08426
    Precipitable_Water  (time, lat, lon) float64 28MB 1.013 1.0 ... 3.242 1.701
    UV_A                (time, lat, lon) float64 28MB 0.0 0.0 ... 0.001347 0.0
    UV_B                (time, lat, lon) float64 28MB 0.0 0.0 ... 0.0001751 0.0
    UVHI                (time, lat, lon) float64 28MB 0.0 0.0 ... 0.001522 0.0

In [6]:
f_regrid = "../temp/WRF_miroc_1985_2014_00054.nc"
ds_rg = xr.open_dataset(f_regrid).drop_dims("bnds")
ds_rg = ds_rg.isel( { "south_north": slice(1, 4), "west_east": slice(2, 5) } )
ds_rg_2 = ds_rg.copy()
ds_rg["XLAT"] = ds_rg["XLAT"].isel({"west_east": 0})
ds_rg["XLONG"] = ds_rg["XLONG"].isel({"south_north": 0})
ds_rg = ds_rg.rename_vars( { "XLAT": "lat",
    "XLONG": "lon", "XTIME": "time" } )
ds_rg = ds_rg.swap_dims( { "south_north": "lat", 
    "west_east": "lon", "XTIME": "time" } )
ds_rg

<xarray.Dataset> Size: 552B
Dimensions:    (time: 12, lon: 3, lat: 3)
Coordinates:
  * time       (time) datetime64[ns] 96B 2014-01-31T23:00:00 ... 2014-12-31T1...
  * lon        (lon) float32 12B -99.17 -99.14 -99.12
  * lat        (lat) float32 12B 19.4 19.42 19.44
Data variables:
    Dew_Point  (time, lat, lon) float32 432B ...
Attributes: (12/141)
    CDI:                             Climate Data Interface version 2.4.0 (ht...
    Conventions:                     CF-1.6
    history:                         Sat Mar 30 03:24:49 2024: cdo -s -w ymon...
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      1984-09-27_00:00:00
    SIMULATION_START_DATE:           1984-01-01_00:00:00
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    CDO:                             Climate Data Operators version 2.4.0 (ht...

In [7]:
# Cambiamos la resolución.
regridder = xe.Regridder( ds_2_corr, ds_rg, "conservative" )
ds_2_rg = regridder( ds_2_corr, keep_attrs = True )
ds_2_rg

<xarray.Dataset> Size: 238MB
Dimensions:             (time: 219144, lat: 3, lon: 3)
Coordinates:
  * time                (time) datetime64[ns] 2MB 1998-01-01 ... 2022-12-31T2...
  * lon                 (lon) float32 12B -99.17 -99.14 -99.12
  * lat                 (lat) float32 12B 19.4 19.42 19.44
Data variables: (12/15)
    Temperature         (time, lat, lon) float64 16MB 284.7 284.5 ... 234.9
    GHI                 (time, lat, lon) float64 16MB 0.0 0.0 ... 63.65 62.41
    DNI                 (time, lat, lon) float64 16MB 0.0 0.0 ... 1.257e+03
    Relative_Humidity   (time, lat, lon) float64 16MB 48.95 49.88 ... 42.48
    Pressure            (time, lat, lon) float64 16MB 788.6 787.1 ... 637.7
    Wind_Speed          (time, lat, lon) float64 16MB 2.718 1.69 ... 0.7701
    ...                  ...
    Solar_Zenith_Angle  (time, lat, lon) float64 16MB 92.81 58.02 ... 66.92
    Surface_Albedo      (time, lat, lon) float64 16MB 0.09535 0.3565 ... 0.09383
    Precipitable_Water  (time, lat, lon) float64 16MB 1.482 3.751 ... 1.987
    UV_A                (time, lat, lon) float64 16MB 0.0 0.0 ... 0.0005298
    UV_B                (time, lat, lon) float64 16MB 0.1965 1.011 ... 6.887e-05
    UVHI                (time, lat, lon) float64 16MB 0.1965 1.011 ... 0.0005987
Attributes:
    regrid_method:  conservative

In [8]:
ds_f = ds_rg_2.reindex( { "XTIME": ds_2_rg["time"].values }
    ).drop_vars("Dew_Point")
dims = ["XTIME", "south_north", "west_east"]

vars = list(ds_2_rg.keys())

for v in vars: ds_f[v] = ( dims, ds_2_rg[v].values )
ds_f[[v]].to_netcdf( path_r + "quantile_prep/NSRDB_int_001.nc" ) 

for i in ["UV_A", "UV_B", "Solar_Zenith_Angle", "Dew_Point", "DHI"]:
    vars.remove(i)

for v in vars:
    if not os.path.exists( path_r + "quantile_vars/" + v + "/" ):
        os.mkdir( path_r + "quantile_vars/" + v + "/" )
    ds_f[[v]].to_netcdf( path_r + "quantile_vars/"
        + v + "/WRF_miroc_1985_2014_00054.nc" ) 